In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SHex
from poly import lagrange_list
from basis import LagrangeBasisHex
from mesh import Mesh, uniform_nodes_3d
from assemble import simple_build_rhs
from cyassemble import simple_assembly
from poisson import poisson_Kloc, poisson_Mloc

In [ ]:
L = 2*np.pi
x_max = L
y_max = L
z_max = L

k1 = 1.0
k2 = 2.0
k3 = 3.0

In [ ]:
def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    sol  = np.sin(k1*2*np.pi*x/x_max)
    sol *= np.sin(k2*2*np.pi*y/y_max)
    sol *= np.sin(k3*2*np.pi*z/z_max)
    return sol.reshape(shape)

def f2(X):
    sol  = -f(X)
    sol *=  (k1*2*np.pi/x_max)**2\
           +(k2*2*np.pi/y_max)**2\
           +(k3*2*np.pi/z_max)**2
    return -sol

In [ ]:
topo = SHex()
errs = {}
order_vals = [1,2,3]
for order in order_vals:
    basis = LagrangeBasisHex(topo, order)
    errs[order] = []
    for n_elems in [8,16]:
        
        vertices, elem_to_vertex, boundary_vertices = \
                 uniform_nodes_3d(n_elems, x_max, y_max, z_max)
        mesh = Mesh(topo, basis)
        mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

        nodes = vertices[elem_to_vertex]
        jacb  = topo.calc_jacb(nodes)
        jacb_det = topo.calc_jacb_det(jacb)
        jacb_inv = topo.calc_jacb_inv(jacb)
        jacb     = jacb[0]
        jacb_det = jacb_det[0]
        jacb_inv = jacb_inv[0]
 
        Kloc = poisson_Kloc(basis, jacb_det, jacb_inv)
        K    = simple_assembly(mesh, Kloc)
        
        Mloc = poisson_Mloc(basis, jacb_det)
        M    = simple_assembly(mesh, Mloc)
        
        rhs = simple_build_rhs(topo, basis, mesh, f2)

        ml = ruge_stuben_solver(K)
        residuals = []
        sol = ml.solve(rhs, tol=1e-12, 
                       residuals=residuals, maxiter=5000)
        
        dof_phys = mesh.get_dof_phys()
        u   = sol-f(dof_phys)
        err = np.sqrt(u.dot(M.dot(u)))
        
        errs[order].append(err)

In [ ]:
for order in order_vals:
    print order, -np.log2(errs[order][1]/errs[order][0])